<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/0709.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.

In [ ]:
import plotly.graph_objects as go
import cv2
import numpy as np
import tempfile

def get_line_pixel_ratio(fig, threshold=250):
    """
    將 plotly figure 存為圖片，讀取後統計灰階像素中非白比例（視為畫線區域）

    參數：
        fig: plotly figure
        threshold: 小於此灰階值視為畫過（0~255）

    回傳：
        line_ratio: 畫線像素 / 總像素
    """
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        fig.write_image(tmpfile.name, width=800, height=400, engine="kaleido")

        # 讀入圖片 → 灰階
        img = cv2.imread(tmpfile.name, cv2.IMREAD_GRAYSCALE)

    # 統計畫線 pixel 占比
    total_pixels = img.size
    line_pixels = np.sum(img < threshold)

    return line_pixels / total_pixels


In [ ]:
methods = {
    'fft': plot_fft_align,
    'template': plot_template_align,
    'dtw': plot_dtw_align
}

for name, plot_func in methods.items():
    fig = plot_func(...)
    ratio = get_line_pixel_ratio(fig)
    print(f"{name}: 線條佔比 = {ratio:.4f}")


### 2.

In [ ]:
def compute_overlay_density(fig, threshold=50):
    """
    將 plotly 圖轉為圖像，計算畫線區域（非背景）像素占比
    threshold: 灰階閾值，小於此值視為背景
    """
    # 1. 儲存 Plotly 為 PNG 圖像 (in memory)
    img_bytes = fig.to_image(format="png", scale=2)
    img = Image.open(io.BytesIO(img_bytes)).convert('L')  # 轉灰階

    # 2. 灰階轉 numpy array
    img_np = np.array(img)

    # 3. 閾值處理：視為畫線區域
    mask = img_np < threshold  # 黑色線條 pixel

    # 4. 計算比例
    painted_ratio = np.sum(mask) / mask.size
    return painted_ratio, img_np, mask

### 3. 哪個方法畫越少點（像素） → 代表線越重疊 → align 越好

In [ ]:
import plotly.graph_objects as go
import cv2
import numpy as np
import tempfile

def get_line_pixel_count(fig, threshold=250):
    """
    將 plotly figure 存為圖片，統計畫線的像素總數（越少代表越集中）

    參數：
        fig: plotly figure
        threshold: 小於此灰階值視為畫線（0~255）# 圖片轉成灰階後，像素值範圍是 0（黑）~ 255（白）

    回傳：
        line_pixel_count: 有畫線的像素數
    """
    with tempfile.NamedTemporaryFile(suffix=".png", delete=False) as tmpfile:
        fig.write_image(tmpfile.name, width=800, height=400, engine="kaleido")
        img = cv2.imread(tmpfile.name, cv2.IMREAD_GRAYSCALE)

    line_pixels = np.sum(img < threshold) # 像素值 < 250 → 視為「有線條」
    return line_pixels


# 使用
fig = plot_aligned_signals(ref_signal, all_signals, ref_job)
line_pixel_count = get_line_pixel_count(fig)

print(f"線條像素數（越少越集中）：{line_pixel_count}")


# 多方法比較
results = {}

for name, plot_func in methods.items():  # methods 是不同 align 畫圖函式
    fig = plot_func(...)
    count = get_line_pixel_count(fig)
    results[name] = count

# 排序看看誰效果最好（線條重疊最多 = 畫線像素最少）
sorted_result = sorted(results.items(), key=lambda x: x[1])
print("\n== Overlay 評分 ==")
for name, count in sorted_result:
    print(f"{name:10s} → 畫線像素：{count}")

### 4. 轉圖片注意事項

In [ ]:
# plot_bgcolor='white'，避免背景深色影響灰階判斷
fig.update_layout(plot_bgcolor='white')

# 取消或調整 legend，會佔用空間並影響像素計算 (若只想比對線條疊合程度，可以先關閉 legend)
fig.update_layout(showlegend=False)
# 或把 legend 放圖外（右側），不影響畫布主區域
fig.update_layout(
    legend=dict(
        x=1.05,
        y=1,
        bgcolor='rgba(255,255,255,0)',
        bordercolor='rgba(0,0,0,0)'
    )
)

# 固定圖像大小
fig.update_layout(width=800, height=400)

# 去除多餘的 margin
fig.update_layout(margin=dict(l=40, r=20, t=40, b=40))

# 若目的是比較 overlay 程度 → 建議去除所有非必要元件
def clean_layout_for_image(fig, width=800, height=400):
    fig.update_layout(
        showlegend=False,
        title='',
        xaxis_title='',
        yaxis_title='',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=10, r=10, t=10, b=10),
        width=width,
        height=height
    )
    return fig